In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
import seaborn as sns
import pandas as pd

In [ ]:
## Load QSO catalog
filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
    'JPAS_mocks_classification_01sep_model11/Fluxes/Qso_jpas_mock_flam_train.cat')

my_filter_order = np.arange(60)
my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
my_filter_order[1:-4] += 1
my_filter_order[12:-4] += 1
my_filter_order[28:-4] += 1
my_filter_order[43:-4] += 1

pm_data = pd.read_csv(
    filename, sep=' ', usecols=range(2, 2 + 60)
).to_numpy().T[my_filter_order]
pm_err = pd.read_csv(
    filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
).to_numpy().T[my_filter_order]
zspec = pd.read_csv(filename, sep=' ', usecols=[127]).to_numpy().reshape(-1, )

w_central = central_wavelength()

In [ ]:
N_sources = pm_data.shape[1]
mag = flux_to_mag(pm_data, w_central[-2])
print(N_sources)

In [ ]:
n_bins = 25
n_steps_ew = 8
ew_min = 0
ew_max = 70
mag_min = 18.18
mag_max = 23
z_min = 2.27
z_max = 4.45
comp_mat_z = np.zeros((n_steps_ew, n_bins - 1))
puri_mat_z = np.copy(comp_mat_z)
puri_mat_rsdss = np.copy(comp_mat_z)
comp_mat_rsdss = np.copy(comp_mat_z)
detec_r = []
detec_z = []
for i, ew0min in enumerate(np.linspace(ew_min, ew_max, n_steps_ew)):
    print('EW0 = {}...'.format(int(ew0min)))
    sel, sel_single, list_lya, _ = QSO_find_lines(
        pm_data, pm_err, 6, 50, ew0min, 15, 6
    )

    sel_mask = np.zeros(N_sources).astype(bool)
    sel_mask[sel_single] = True

    z_nb_Arr = np.ones(N_sources) * -1
    for src in sel_single:
        z_nb_Arr[src] = w_central[list_lya[src]] / 1215.67 - 1

    # Histograms as a function of z
    bins = np.linspace(z_min, z_max, n_bins)
    h_zspec, b = np.histogram(zspec[zspec > 2.5], bins=bins)
    h_detec_z, _ = np.histogram(zspec[sel_mask], bins=bins)
    h_detec_r_z, _ = np.histogram(zspec[sel_mask & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)

    # Histograms as a function of rSDSS
    bins = np.linspace(mag_min, mag_max, n_bins)
    h_rsdss, b = np.histogram(mag[-2, (zspec > 2.5) & (zspec < 4.5)], bins=bins)
    h_detec_rsdss, _ = np.histogram(mag[-2, sel_mask], bins=bins)
    h_detec_r_rsdss, _ = np.histogram(mag[-2, sel_mask & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)
    
    comp_mat_rsdss[i] = h_detec_r_rsdss / h_rsdss
    puri_mat_rsdss[i] = h_detec_r_rsdss / h_detec_rsdss
    comp_mat_z[i] = h_detec_r_z / h_zspec
    puri_mat_z[i] = h_detec_r_z / h_detec_z

    detec_z.append(h_detec_z)
    detec_r.append(h_detec_rsdss)
detec_z = np.array(detec_z)
detec_r = np.array(detec_r)

In [ ]:
# COMPLETENESS z
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5),
    gridspec_kw={'width_ratios': [4, 1]})
plt.subplots_adjust(wspace=-0.1)

xticks = ['{0:0.2f}'.format(n) for n in np.linspace(z_min, z_max, n_bins - 1)]
yticks = ['{0:0.0f}'.format(n) for n in np.linspace(ew_min, ew_max, n_steps_ew)]

comp_mat_z[np.isinf(comp_mat_z)] = 0
comp_mat_z[np.isnan(comp_mat_z)] = 0
sns.heatmap(comp_mat_z, vmin=0, vmax=1, ax=ax0)
ax0.set_xticklabels(xticks, rotation='vertical')
ax0.set_yticklabels(yticks, rotation='horizontal')
ax0.set_xlabel('Redshift', fontsize=15)
ax0.set_ylabel('Minimum EW$_0$', fontsize=15)
ax0.invert_yaxis()
ax0.set_title('Completeness (z)', fontsize=20)

ax1.plot(np.sum(comp_mat_z * h_zspec, axis=1) / np.sum(h_zspec),
    np.linspace(ew_min, ew_max, n_steps_ew))
ax1.set_ylabel('Minimum EW$_0$', fontsize=15)
ax1.set_xlabel('Total completeness', fontsize=15)
ax1.yaxis.set_label_position("right")
ax1.yaxis.tick_right()
plt.show()

# PURITY z
fig, (ax0, ax1)= plt.subplots(1, 2, figsize=(10, 5),
    gridspec_kw={'width_ratios': [4, 1]})
plt.subplots_adjust(wspace=-0.1)

xticks = ['{0:0.2f}'.format(n) for n in np.linspace(z_min, z_max, n_bins - 1)]
yticks = ['{0:0.0f}'.format(n) for n in np.linspace(ew_min, ew_max, n_steps_ew)]

puri_mat_z[np.isnan(puri_mat_z)] = 0
sns.heatmap(puri_mat_z, vmin=0, vmax=1, ax=ax0)
ax0.set_xticklabels(xticks, rotation='vertical')
ax0.set_yticklabels(yticks, rotation='horizontal')
ax0.set_xlabel('Redshift', fontsize=15)
ax0.set_ylabel('Minimum EW$_0$', fontsize=15)
ax0.invert_yaxis()
ax0.set_title('Purity (z)', fontsize=20)

ax1.plot(np.sum(puri_mat_z * detec_z, axis=1) / np.sum(detec_z, axis=1),
    np.linspace(ew_min, ew_max, n_steps_ew))
ax1.set_ylabel('Minimum EW$_0$', fontsize=15)
ax1.set_xlabel('Total Purity', fontsize=15)
ax1.yaxis.set_label_position("right")
ax1.yaxis.tick_right()
plt.show()

# COMPLETENESS r
fig, (ax0, ax1)= plt.subplots(1, 2, figsize=(10, 5),
    gridspec_kw={'width_ratios': [4, 1]})
plt.subplots_adjust(wspace=-0.1)

xticks = ['{0:0.2f}'.format(n) for n in np.linspace(mag_min, mag_max, n_bins - 1)]
yticks = ['{0:0.0f}'.format(n) for n in np.linspace(ew_min, ew_max, n_steps_ew)]

comp_mat_rsdss[np.isinf(comp_mat_rsdss)] = 0
comp_mat_rsdss[np.isnan(comp_mat_rsdss)] = 0
sns.heatmap(comp_mat_rsdss, vmin=0, vmax=1, ax=ax0)
ax0.set_xticklabels(xticks, rotation='vertical')
ax0.set_yticklabels(yticks, rotation='horizontal')
ax0.set_xlabel('r', fontsize=15)
ax0.set_ylabel('Minimum EW$_0$', fontsize=15)
ax0.invert_yaxis()
ax0.set_title('Completeness (rSDSS)', fontsize=20)

ax1.plot(np.sum(comp_mat_rsdss * h_rsdss, axis=1) / np.sum(h_rsdss),
    np.linspace(ew_min, ew_max, n_steps_ew))
ax1.set_ylabel('Minimum EW$_0$', fontsize=15)
ax1.set_xlabel('Total completeness', fontsize=15)
ax1.yaxis.set_label_position("right")
ax1.yaxis.tick_right()
plt.show()

# PURITY r
fig, (ax0, ax1)= plt.subplots(1, 2, figsize=(10, 5),
    gridspec_kw={'width_ratios': [4, 1]})
plt.subplots_adjust(wspace=-0.1)

xticks = ['{0:0.2f}'.format(n) for n in np.linspace(mag_min, mag_max, n_bins - 1)]
yticks = ['{0:0.0f}'.format(n) for n in np.linspace(ew_min, ew_max, n_steps_ew)]

puri_mat_rsdss[np.isnan(puri_mat_rsdss)] = 0
sns.heatmap(puri_mat_rsdss, vmin=0, vmax=1, ax=ax0)
ax0.set_xticklabels(xticks, rotation='vertical')
ax0.set_yticklabels(yticks, rotation='horizontal')
ax0.set_xlabel('r', fontsize=15)
ax0.set_ylabel('Minimum EW$_0$', fontsize=15)
ax0.invert_yaxis()
ax0.set_title('Purity (rSDSS)', fontsize=20)

ax1.plot(np.sum(puri_mat_rsdss * detec_r, axis=1) / np.sum(detec_r, axis=1),
    np.linspace(ew_min, ew_max, n_steps_ew))
ax1.set_ylabel('Minimum EW$_0$', fontsize=15)
ax1.set_xlabel('Total Purity', fontsize=15)
ax1.yaxis.set_label_position("right")
ax1.yaxis.tick_right()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

sns.heatmap(detec_r, ax=ax)
ax.set_xticklabels(xticks, rotation='vertical')
ax.set_yticklabels(yticks, rotation='horizontal')
ax.set_xlabel('rSDSS', fontsize=15)
ax.set_ylabel('EW$_0$ min ($\AA$)', fontsize=15)
plt.show()

fig, ax = plt.subplots(figsize=(8, 6))

xticks = ['{0:0.2f}'.format(n) for n in np.linspace(z_min, z_max, n_bins - 1)]
yticks = ['{0:0.0f}'.format(n) for n in np.linspace(ew_min, ew_max, n_steps_ew)]

sns.heatmap(detec_z, ax=ax)
ax.set_xticklabels(xticks, rotation='vertical')
ax.set_yticklabels(yticks, rotation='horizontal')
ax.set_xlabel('z', fontsize=15)
ax.set_ylabel('EW$_0$ min ($\AA$)', fontsize=15)
plt.show()

In [ ]:
comp_mat_z = np.zeros((n_steps_ew, n_bins - 1))
puri_mat_z = np.copy(comp_mat_z)
puri_mat_rsdss = np.copy(comp_mat_z)
comp_mat_rsdss = np.copy(comp_mat_z)
comp_mat_z_s = np.zeros((n_steps_ew, n_bins - 1))
puri_mat_z_s = np.copy(comp_mat_z)
puri_mat_rsdss_s = np.copy(comp_mat_z)
comp_mat_rsdss_s = np.copy(comp_mat_z)
detec_r = []
detec_z = []
detec_r_s = []
detec_z_s = []
for i, ew0min in enumerate(np.linspace(ew_min, ew_max, n_steps_ew)):
    print('EW0 = {}...'.format(int(ew0min)))
    sel, sel_single, list_lya, _ = QSO_find_lines(
        pm_data, pm_err, 6, 50, ew0min, 15, 6
    )

    sel_mask = np.zeros(N_sources).astype(bool)
    sel_mask_s = np.zeros(N_sources).astype(bool)
    sel_mask[sel] = True
    sel_mask_s[sel_single] = True

    z_nb_Arr = np.ones(N_sources) * -1
    for src in sel_single:
        z_nb_Arr[src] = w_central[list_lya[src]] / 1215.67 - 1
    for src in sel:
        z_nb_Arr[src] = w_central[list_lya[src]] / 1215.67 - 1

    # Histograms as a function of z
    bins = np.linspace(z_min, z_max, n_bins)
    h_zspec, b = np.histogram(zspec[(zspec > z_min) & (zspec < z_max)], bins=bins)
    h_detec_z, _ = np.histogram(zspec[sel_mask], bins=bins)
    h_detec_r_z, _ = np.histogram(zspec[sel_mask & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)
    h_detec_z_s, _ = np.histogram(zspec[sel_mask_s], bins=bins)
    h_detec_r_z_s, _ = np.histogram(zspec[sel_mask_s & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)

    # Histograms as a function of rSDSS
    bins = np.linspace(mag_min, mag_max, n_bins)
    h_rsdss, b = np.histogram(mag[-2, (zspec > z_min) & (zspec < z_max)], bins=bins)
    h_detec_rsdss, _ = np.histogram(mag[-2, sel_mask], bins=bins)
    h_detec_r_rsdss, _ = np.histogram(mag[-2, sel_mask & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)
    h_detec_rsdss_s, _ = np.histogram(mag[-2, sel_mask_s], bins=bins)
    h_detec_r_rsdss_s, _ = np.histogram(mag[-2, sel_mask_s & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)

    comp_mat_rsdss[i] = h_detec_r_rsdss / h_rsdss
    puri_mat_rsdss[i] = h_detec_r_rsdss / h_detec_rsdss
    comp_mat_z[i] = h_detec_r_z / h_zspec
    puri_mat_z[i] = h_detec_r_z / h_detec_z

    comp_mat_rsdss_s[i] = h_detec_r_rsdss_s / h_rsdss
    puri_mat_rsdss_s[i] = h_detec_r_rsdss_s / h_detec_rsdss_s
    comp_mat_z_s[i] = h_detec_r_z_s / h_zspec
    puri_mat_z_s[i] = h_detec_r_z_s / h_detec_z_s

    detec_z.append(h_detec_z)
    detec_r.append(h_detec_rsdss)
    detec_z_s.append(h_detec_z_s)
    detec_r_s.append(h_detec_rsdss_s)
detec_z = np.array(detec_z)
detec_r = np.array(detec_r)
detec_z_s = np.array(detec_z_s)
detec_r_s = np.array(detec_r_s)comp_mat_z = np.zeros((n_steps_ew, n_bins - 1))
puri_mat_z = np.copy(comp_mat_z)
puri_mat_rsdss = np.copy(comp_mat_z)
comp_mat_rsdss = np.copy(comp_mat_z)
comp_mat_z_s = np.zeros((n_steps_ew, n_bins - 1))
puri_mat_z_s = np.copy(comp_mat_z)
puri_mat_rsdss_s = np.copy(comp_mat_z)
comp_mat_rsdss_s = np.copy(comp_mat_z)
detec_r = []
detec_z = []
detec_r_s = []
detec_z_s = []
for i, ew0min in enumerate(np.linspace(ew_min, ew_max, n_steps_ew)):
    print('EW0 = {}...'.format(int(ew0min)))
    sel, sel_single, list_lya, _ = QSO_find_lines(
        pm_data, pm_err, 6, 50, ew0min, 15, 6
    )

    sel_mask = np.zeros(N_sources).astype(bool)
    sel_mask_s = np.zeros(N_sources).astype(bool)
    sel_mask[sel] = True
    sel_mask_s[sel_single] = True

    z_nb_Arr = np.ones(N_sources) * -1
    for src in sel_single:
        z_nb_Arr[src] = w_central[list_lya[src]] / 1215.67 - 1
    for src in sel:
        z_nb_Arr[src] = w_central[list_lya[src]] / 1215.67 - 1

    # Histograms as a function of z
    bins = np.linspace(z_min, z_max, n_bins)
    h_zspec, b = np.histogram(zspec[(zspec > z_min) & (zspec < z_max)], bins=bins)
    h_detec_z, _ = np.histogram(zspec[sel_mask], bins=bins)
    h_detec_r_z, _ = np.histogram(zspec[sel_mask & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)
    h_detec_z_s, _ = np.histogram(zspec[sel_mask_s], bins=bins)
    h_detec_r_z_s, _ = np.histogram(zspec[sel_mask_s & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)

    # Histograms as a function of rSDSS
    bins = np.linspace(mag_min, mag_max, n_bins)
    h_rsdss, b = np.histogram(mag[-2, (zspec > z_min) & (zspec < z_max)], bins=bins)
    h_detec_rsdss, _ = np.histogram(mag[-2, sel_mask], bins=bins)
    h_detec_r_rsdss, _ = np.histogram(mag[-2, sel_mask & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)
    h_detec_rsdss_s, _ = np.histogram(mag[-2, sel_mask_s], bins=bins)
    h_detec_r_rsdss_s, _ = np.histogram(mag[-2, sel_mask_s & (np.abs(zspec - z_nb_Arr) < 0.1)],
        bins=bins)

    comp_mat_rsdss[i] = h_detec_r_rsdss / h_rsdss
    puri_mat_rsdss[i] = h_detec_r_rsdss / h_detec_rsdss
    comp_mat_z[i] = h_detec_r_z / h_zspec
    puri_mat_z[i] = h_detec_r_z / h_detec_z

    comp_mat_rsdss_s[i] = h_detec_r_rsdss_s / h_rsdss
    puri_mat_rsdss_s[i] = h_detec_r_rsdss_s / h_detec_rsdss_s
    comp_mat_z_s[i] = h_detec_r_z_s / h_zspec
    puri_mat_z_s[i] = h_detec_r_z_s / h_detec_z_s

    detec_z.append(h_detec_z)
    detec_r.append(h_detec_rsdss)
    detec_z_s.append(h_detec_z_s)
    detec_r_s.append(h_detec_rsdss_s)
detec_z = np.array(detec_z)
detec_r = np.array(detec_r)
detec_z_s = np.array(detec_z_s)
detec_r_s = np.array(detec_r_s)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
mag[np.isnan(mag)] = 99
ax.hist2d(zspec, mag[-2], bins=[np.linspace(0, 4, 30), np.linspace(19, 25, 30)])
plt.show()